# IMPORTS

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import logging
import typing
import gc
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
import torch_xla
from torch import nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch_xla.core.xla_model as xm

# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.
from src.asrsnlp.pipelines.train_bert import nodes
from src.asrsnlp.pipelines import feature_engineering

/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/__init__.py:44: KedroDeprecationWarning: 'AbstractDataSet' has been renamed to 'AbstractDataset', and the alias will be removed in Kedro 0.19.0
  return getattr(kedro.io.core, name)
/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/__init__.py:44: KedroDeprecationWarning: 'DataSetError' has been renamed to 'DatasetError', and the alias will be removed in Kedro 0.19.0
  return getattr(kedro.io.core, name)
/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro_mlflow/io/models/mlflow_model_logger_dataset.py:4: KedroDeprecationWarning: 'DataSetError' has been renamed to 'DatasetError', and the alias will be removed in Kedro 0.19.0
  from kedro.io.core import DataSetError
/home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/__init__.py:44: KedroDeprecationWarning: 'AbstractVersionedDataSet' has

In [2]:
pwd

'/home/ensai/dev/asrsnlp'

In [5]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()

In [6]:
modelname = catalog.load("params:bert_params.name")
llayer = catalog.load("params:bert_params.llayer")
dropoutratio = catalog.load("params:bert_params.doratio")
max_len = 200
trainparams = catalog.load("params:berttrain_params")

[12/01/23 12:50:24] INFO     Loading data from 'params:bert_params.name' (MemoryDataset)...     ]8;id=318728;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=323041;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:bert_params.llayer' (MemoryDataset)...   ]8;id=177346;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=420268;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:bert_params.doratio' (MemoryDataset)...  ]8;id=640558;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=365295;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:berttrain_params' (MemoryDataset)...     ]8;id=187016;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=757881;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [7]:
model = nodes.ModelClass(modelname, (768,15), dropoutratio)

In [8]:
tokenizer, model = nodes.import_model(modelname, llayer, dropoutratio)

In [9]:
train_feat = catalog.load("train_cleaned_fea")
test_feat = catalog.load("test_cleaned_fea")

[12/01/23 12:50:36] INFO     Loading data from 'train_cleaned_fea' (MlflowParquetDataset)...    ]8;id=478170;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=47738;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[12/01/23 12:50:37] INFO     Loading data from 'test_cleaned_fea' (MlflowParquetDataset)...     ]8;id=14995;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=514473;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [10]:
train_loaded = nodes.loader(train_feat, tokenizer, max_len, trainparams, sample=1)
test_loaded = nodes.loader(test_feat, tokenizer, max_len, trainparams, sample=1)

In [11]:
pos_num = torch.zeros(15)
for _, data in enumerate(train_loaded, 0) :
    targets = data['targets']
    pos_num += torch.sum(targets, axis=0)
nobs = len(train_loaded.dataset)
pos_weight = (nobs - pos_num) / pos_num

In [64]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [65]:
t1 = torch.tensor([1,2]).to(device)
t2 = torch.tensor([1,2])
t1 + t2

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 t1 = torch.tensor([1,2]).to(device)                                                          │
│   2 t2 = torch.tensor([1,2])                                                                     │
│ ❱ 3 t1 + t2                                                                                      │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [13]:
loss1 = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
loss2 = torch.nn.BCEWithLogitsLoss()

In [14]:
labels = catalog.load('params:labels')

[12/01/23 12:53:25] INFO     Loading data from 'params:labels' (MemoryDataset)...               ]8;id=726963;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=785096;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [28]:
labels_with_other = labels+['Other']
colors = px.colors.qualitative.Set2[:len(labels_with_other)]

In [33]:
px.bar(x = labels_with_other,
       y=pos_num,
       width=1000,
       height=500,
       labels={'x':'Anomalies', 'y':'Occurrences', 'title_text': 'Labels Occurences'},
       color=labels_with_other,
       color_discrete_sequence=colors).update_layout(title_text='Labels Occurrences')

In [35]:
px.bar(x = labels_with_other,
       y=pos_weight,
       width=1000,
       height=500,
       labels={'x':'Anomalies', 'y':'Occurrences'},
       color=labels_with_other,
       color_discrete_sequence=colors).update_layout(title_text='Labels Weights')

In [37]:
px.bar(x = labels_with_other,
       y=pos_weight * pos_num,
       width=1000,
       height=500,
       labels={'x':'Anomalies', 'y':'Occurrences'},
       color=labels_with_other,
       color_discrete_sequence=colors).update_layout(title_text='Balanced Occurences')

In [15]:
device = xm.xla_device()

[12/01/23 03:00:14] WARNING  PJRT is now the default runtime. For more information, see               ]8;id=45896;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=749828;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#35\35]8;;\
                             https://github.com/pytorch/xla/blob/master/docs/pjrt.md                               

                    WARNING  Defaulting to PJRT_DEVICE=CPU                                            ]8;id=218722;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=31413;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#46\46]8;;\

In [16]:
ids, mask, token_type_ids, targets = next(iter(train_loaded)).values()

In [17]:
ids.to(device, dtype=torch.long)
mask.to(device, dtype=torch.long)
token_type_ids.to(device, dtype=torch.long)
targets.to(device, dtype=torch.long)


tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]], device='xla:0')

In [18]:
model.to(device)


ModelClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [19]:
output = model(ids, mask, token_type_ids)

In [21]:
loss1(output, targets)


_to_functional_tensor(tensor(2.9030),
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [22]:
loss2(output, targets)


_to_functional_tensor(tensor(0.7173),
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [66]:
def get_device():
    """return available device
    """
    device = xm.xla_device()
    if 'TPU' not in str(xm.xla_real_devices(devices=[device])):
        if torch.cuda.is_available() :
            device = torch.device('cuda')
            try:
                _ = torch.tensor([1, 2]).to(device) + torch.tensor([1, 2]).to(device)
            except RuntimeError:
                device = torch.device('cuda:0')
            torch.cuda.empty_cache()
            os.environ['PJRT_DEVICE'] = 'GPU'
    return device

In [67]:
get_device()

[12/01/23 18:15:47] WARNING  PJRT is now the default runtime. For more information, see               ]8;id=462156;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=981736;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#35\35]8;;\
                             https://github.com/pytorch/xla/blob/master/docs/pjrt.md                               

                    WARNING  Defaulting to PJRT_DEVICE=CPU                                            ]8;id=896876;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=935517;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#46\46]8;;\

device(type='cuda')